In [33]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [34]:
import re

In [73]:
from tqdm import tqdm_notebook

In [61]:
import numpy as np

In [55]:
def ExtractDetails(url):
    Details = {}
    response = requests.get(url)
    PgSrc = BeautifulSoup(response.text,'lxml')
    
    Details['Title'] = PgSrc.find('h1',class_ = "title page-title").text
    Misc = PgSrc.find('div',id = "metaabstract")
    Details['Topic'] = Misc.find('div',class_ = 'field field--name-field-abstract-topic field--type-string field--label-hidden field__item').text.replace('\n','')

    Details['Poster/Abstract Number'] = Misc.find('div',class_ = 'field field--name-field-assigned-number field--type-string field--label-hidden field__item').text.replace('\n','')
    Details['Authors'] = Misc.find('div',class_ = 'field field--name-field-co-authors field--type-string field--label-hidden field__items').text.replace('\n','')
#     dno = 1
#     for det in MiscDet:
#         Details['Detail number '+str(dno)] = det.text
#         dno+=1
    Details['Organisations/Institutions'] = PgSrc.find('details',id = 'institutions').text.replace('\n','')
    Details['Summary'] = PgSrc.find('div',class_ = 'clearfix text-formatted field field--name-field-mt-prs-body field--type-text-with-summary field--label-hidden field__item').text.replace('\n','')
    return Details

In [56]:
Details = ExtractDetails('https://mdaconference.org/node/920')
Details

{'Title': 'A biomarker of DUX4 activity to evaluate losmapimod treatment effect in FSHD Phase 2 trials\n',
 'Topic': ' Clinical Trials',
 'Poster/Abstract Number': '30',
 'Authors': 'Lucienne Ronco, PhD, Kathryn Wagner, MD, Diego Cadavid, MD, Jayashree Chadchankar, PhD, Aaron Chang, PhD, Michelle Mellion, MD, Shane Raines, PhD, Anugraha Raman, PhD, Alan Robertson, MS, Alejandro Rojas, PhD, Ning Shen, PhD, Jeffrey Statland, MD, Rabi Tawil, MD, Stephen Tapscott, MD, PhD, Baziel van Engelen, MD, Leo Wang, MD, Nicholas Johnson, MD, Owen Wallace, PhD',
 'Organisations/Institutions': ' Organizations/Institutions1. Fulcrum Therapeutics, 2. Kennedy Krieger Institute, Johns Hopkins School of Medicine, Baltimore, MD, USA, 3. Fulcrum Therapeutics, 5. Fulcrum Therapeutics, 6. Fulcrum Therapeutics, 9. Fulcrum Therapeutics, 10. Fulcrum Therapeutics, 11. Fulcrum Therapeutics, 12. University of Kansas Medical Center, 13. University of Rochester, 14. Human Biology Division, Fred Hutchinson Cancer Resea

In [53]:
def ExtractLinks(url):
    Links = []
    response = requests.get(url)
    data = BeautifulSoup(response.text,'lxml')
    Abstracts = data.find_all('div',class_ = re.compile('abstractsearchcol views-col col-'))
    for abstract in Abstracts:
        Links.append('https://mdaconference.org' + abstract.find('a')['href'])
    return Links

In [48]:
Links = []
Links = ExtractLinks('https://mdaconference.org/abstract-search?combine=&field_assigned_number_value=&field_co_authors_value=&field_abstract_topic_value=All&page=25')

In [147]:
MasterLinks = []
Links = ExtractLinks('https://mdaconference.org/abstract-search?combine=&field_assigned_number_value=&field_co_authors_value=&field_abstract_topic_value=All&page=0')
dfDetails = pd.DataFrame()
i = 0
while len(Links)!=0:
    MasterLinks.append(Links) # Could have just gone with list.extend rather than hstack
    i+=1
    url = 'https://mdaconference.org/abstract-search?combine=&field_assigned_number_value=&field_co_authors_value=&field_abstract_topic_value=All&page=' + str(i)
    Links = ExtractLinks(url)
MasterLinks = np.hstack(MasterLinks)
for link in tqdm_notebook(MasterLinks):
    Details = ExtractDetails(link)
    dfDetails = dfDetails.append(Details,ignore_index = True)

In [84]:
MasterLinks


array(['https://mdaconference.org/node/933',
       'https://mdaconference.org/node/909',
       'https://mdaconference.org/node/1070',
       'https://mdaconference.org/node/1058',
       'https://mdaconference.org/node/898',
       'https://mdaconference.org/node/950',
       'https://mdaconference.org/node/1080',
       'https://mdaconference.org/node/921',
       'https://mdaconference.org/node/1061',
       'https://mdaconference.org/node/1055',
       'https://mdaconference.org/node/1034',
       'https://mdaconference.org/node/923',
       'https://mdaconference.org/node/937',
       'https://mdaconference.org/node/965',
       'https://mdaconference.org/node/979',
       'https://mdaconference.org/node/907',
       'https://mdaconference.org/node/927',
       'https://mdaconference.org/node/1015',
       'https://mdaconference.org/node/948',
       'https://mdaconference.org/node/983',
       'https://mdaconference.org/node/986',
       'https://mdaconference.org/node/1039',
  

In [86]:
Links = ExtractLinks('https://mdaconference.org/abstract-search?combine=&field_assigned_number_value=&field_co_authors_value=&field_abstract_topic_value=All&page=1')
MasterLinks = []
# dfDetails = pd.DataFrame()
i = 1
while len(Links)!=0:
    MasterLinks.append(Links) # Could have just gone with list.extend rather than hstack
    i+=1
    url = 'https://mdaconference.org/abstract-search?combine=&field_assigned_number_value=&field_co_authors_value=&field_abstract_topic_value=All&page=' + str(i)
    Links = ExtractLinks(url)
MasterLinks = np.hstack(MasterLinks)

In [88]:
len(MasterLinks)

177

In [148]:
dfDetails[dfDetails['Poster/Abstract Number'] == '30']

,Authors,Organisations/Institutions,Poster/Abstract Number,Summary,Title,Topic
0,"Lucienne Ronco, PhD, Kathryn Wagner, MD, Diego...",Organizations/Institutions1. Fulcrum Therapeu...,30,Objective: To identify a panel of stable DUX4-...,A biomarker of DUX4 activity to evaluate losma...,Clinical Trials


In [149]:
RearrangedColumns = ['Title','Authors', 'Organisations/Institutions', 'Poster/Abstract Number',
       'Summary',  'Topic']

In [150]:
dfDetails = dfDetails[RearrangedColumns]

In [152]:
dfDetails.to_excel('MDAConference2020.xls')